In [3]:
import sys, os, io, json, numpy as np, random
#from sklearn.metrics import roc_auc_score
#import xgboost as xgb
import tensorflow as tf
import pandas as pd

In [4]:
pool = np.load('../prepare_dataset/pool_00.npz')
features_bool, f_time, label = (pool[name] for name in ['features', 'f_time', 'label'])

In [5]:
features = np.random.normal(loc=features_bool*1.0, scale=1.0)

In [25]:
def create_graph():
    graph = tf.Graph()
    with graph.as_default():
        features = tf.placeholder(dtype=tf.float64)
        label = tf.placeholder(dtype=tf.float64)
        bias = tf.placeholder(dtype=tf.float64)
        ax = tf.transpose(tf.nn.top_k(-tf.transpose(features), k = tf.shape(features)[-2]).indices)
        y = tf.gather(label, ax)[:, :, 0]
        b = tf.gather(bias, ax)[:, :, 0]
        #T = tf.gather(features, ax, axis=0)
        T = tf.contrib.framework.sort(features, axis=0)
        current_loss = tf.reduce_sum(-(label * tf.log_sigmoid(bias) + (1 - label) * tf.log_sigmoid(-bias)), axis=0)
        l_curr = -(y * tf.log_sigmoid(b) + (1 - y) * tf.log_sigmoid(-b))
        l_der1 = -y * tf.sigmoid(-b) + (1 - y) * tf.sigmoid(b)
        l_der2 = tf.sigmoid(-b) * tf.sigmoid(b)
        cum_l = tf.cumsum(l_curr)[:-1, :]  # -1
        cum_l_der1 = tf.cumsum(l_der1)[:-1, :]  # -1
        cum_l_der2 = tf.cumsum(l_der2)[:-1, :]  # -1
        rev_cum_l = tf.cumsum(l_curr, reverse=True)[1:, :]  # -1
        rev_cum_l_der1 = tf.cumsum(l_der1, reverse=True)[1:, :] # -1
        rev_cum_l_der2 = tf.cumsum(l_der2, reverse=True)[1:, :] # -1
        delta_up = -cum_l_der1 / cum_l_der2  # -1
        delta_down = -rev_cum_l_der1 / rev_cum_l_der2 # -1
        loss_up = cum_l + 0.5 * delta_up * cum_l_der1
        loss_down = rev_cum_l + 0.5 * delta_down * rev_cum_l_der1
        whole_loss = (loss_up + loss_down)
        features_amount = tf.to_double(tf.shape(features)[0])
        #######
        #-----#
        #-----#   -->   iiiIiii (best_loss_argmin0), mmmMmmm (min_loss_axis0)
        #-----#
        #######
        best_loss_argmin0 = tf.argmin(whole_loss, axis=0)
        min_loss_axis0 = tf.reduce_min(whole_loss, axis=0)
        # mmmMmmm  ->    I (best_loss_argmin1), M (best_loss)
        best_loss_argmin1 = tf.argmin(min_loss_axis0, axis=0)
        best_loss = tf.reduce_min(min_loss_axis0, axis=0)
        best_avg_loss = best_loss/features_amount
        avg_current_loss = current_loss/features_amount
        # iiiIiii  ->  I
        best_index = best_loss_argmin0[best_loss_argmin1]
        thr = (T[best_index, best_loss_argmin1] + T[best_index+1, best_loss_argmin1])/2
        best_delta_up = delta_up[best_index, best_loss_argmin1]
        best_delta_down = delta_down[best_index, best_loss_argmin1]
        
    return {'graph': graph,
            'features': features,
            'bias': bias, 
            'label': label,
            'l_curr': l_curr,
            'l_der1': l_der1,
            'l_der2': l_der2,
            'cum_l_der1': cum_l_der1,
            'cum_l_der2': cum_l_der2,
            'rev_cum_l_der1': rev_cum_l_der1,
            'rev_cum_l_der2': rev_cum_l_der2,
            'delta_up': delta_up, 
            'delta_down': delta_down,
            'best_delta_up': best_delta_up, 
            'best_delta_down': best_delta_down,
            'cum_l': cum_l,
            'rev_cum_l': rev_cum_l,
            'whole_loss': whole_loss,
            'best_loss_argmin0' : best_loss_argmin0,
            'min_loss_axis0': min_loss_axis0,
            'best_loss_argmin1': best_loss_argmin1,
            'best_feature_index': best_loss_argmin1,
            'best_loss': best_loss,
            'best_avg_loss': best_avg_loss,
            'best_index': best_index,
            'T': T,
            'thr': thr,
            'y': y, 
            'current_loss': current_loss,
            'avg_current_loss': avg_current_loss
           }

In [26]:
split_instances = create_graph()

In [88]:
def make_split(split_instances, bias, features, label):
    graph = split_instances['graph']
    input_tensors = {split_instances[t]: val for t, val in [('features', features), ('bias', bias), ('label', label)]} 
    tensors = {t: split_instances[t] 
               for t in ['thr', 'best_loss', 'best_index', 'best_delta_up', 'best_delta_down', 'current_loss',
                         'best_feature_index',
                        'avg_current_loss', 'best_avg_loss']}
    with tf.Session(graph=graph) as s:
        tensors_values = s.run(tensors, input_tensors)
    return tensors_values

In [42]:
#features_swap = features.copy()
#features_swap[:,0],features_swap[:,7] = features_swap[:,7],features_swap[:,0]
features_swap = np.concatenate([features[:,7:8], features[:,1:7], features[:,0:1], features_swap[:,8:]], axis=1)

In [45]:
%%time
make_split(split_instances, np.zeros(label.shape), features, label)

{'thr': 0.8994369366254575, 'best_loss': 183719.79165202889, 'best_index': 221457, 'best_delta_up': -0.7877611104588681, 'best_delta_down': -0.847139110284943, 'current_loss': array([207944.15416803]), 'best_feature_index': 0, 'avg_current_loss': array([0.69314718]), 'best_avg_loss': 0.6123993055067629}
CPU times: user 5.53 s, sys: 559 ms, total: 6.09 s
Wall time: 2.13 s


In [46]:
%%time
make_split(split_instances, np.zeros(label.shape), features_swap, label)

{'thr': 0.8994369366254575, 'best_loss': 183719.79165202889, 'best_index': 221457, 'best_delta_up': -0.7877611104588681, 'best_delta_down': -0.847139110284943, 'current_loss': array([207944.15416803]), 'best_feature_index': 7, 'avg_current_loss': array([0.69314718]), 'best_avg_loss': 0.6123993055067629}
CPU times: user 5.41 s, sys: 547 ms, total: 5.96 s
Wall time: 2.26 s


In [71]:
class EMatrix:
    def __init__(self, bias, features, label):
        self.bias = bias
        self.features = features
        self.label = label

In [72]:
class LeafData:
    def __init__(self, info):
        self.val = info['prediction']

class SplitData:
    def __init__(self, val):
        self.val = val

class TreeNode:
    def __init__(self):
        self.left = None
        self.right = None
        self.depth = 0
        self.val = None
        

In [82]:
def prior_finish(params, info, parent):
    if parent is None:
        return False
    return params['max_depth'] <= parent.depth

def post_finish(params, info, split_info, parent):
    return False

In [83]:
start_params = {'max_depth': 4}

In [96]:
def split_ematrix(split_instances, ematrix):
    split_info = make_split(split_instances, ematrix.bias, ematrix.features, ematrix.label)
    thr = split_info['thr']
    features = ematrix.features
    bias = ematrix.bias
    label = ematrix.label
    best_feature = features[:, split_info['best_feature_index']] < thr
    best_feature_not = np.logical_not(best_feature)
    left_ematrix = EMatrix(bias[best_feature], features[best_feature, :], label[best_feature])
    right_ematrix = EMatrix(bias[best_feature_not], features[best_feature_not, :], label[best_feature_not])
    left_info = {'prediction': split_info['best_delta_up'], 'ematrix': left_ematrix}
    right_info = {'prediction': split_info['best_delta_down'], 'ematrix': right_ematrix}
    return left_info, right_info, split_info

In [97]:
def build_tree_helper(split_instances, params, info, parent):
    print(parent.depth if parent else '---', file=sys.stderr)
    if prior_finish(params, info, parent):
        return LeafData(info)
    left_info, right_info, split_info = split_ematrix(split_instances, info['ematrix'])
    if post_finish(params, info, split_info, parent):
        return LeafData(info)
    node = TreeNode()
    node.depth = parent.depth + 1 if parent else 0
    node.val = split_info
    node.left = build_tree_helper(split_instances, params, left_info, node)
    node.right = build_tree_helper(split_instances, params, right_info, node)    

In [98]:
def build_tree(split_instances, params, ematrix):
    info = {'ematrix': ematrix}
    return build_tree_helper(split_instances, params, info=info, parent=None)

In [99]:
!git status

On branch master
Your branch is up-to-date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   tf_split.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	../.DS_Store
	../ds.txt
	.ipynb_checkpoints/
	convert_dataset.ipynb
	split.ipynb
	../prepare_dataset/.ipynb_checkpoints/
	../prepare_dataset/pool_00.npz

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
! git a

In [95]:
tree = build_tree(split_instances, start_params, EMatrix(np.zeros(label.shape), features, label))

---


UnboundLocalError: local variable 'best_feature_not' referenced before assignment

In [262]:
test_val.shape, test_val2.shape, T_v.shape, l_curr_v.shape, whole_loss_v.shape

((15,), (15,), (300000, 15), (300000, 15), (299999, 15))

In [263]:
thr_v, bloss_v, bind_v, np.log(2)

(1.3816564892782939, 0.6124175966710163, 256994, 0.6931471805599453)

In [257]:
test_val, test_val2

(array([256994, 248036, 283538, 143258, 147744,  94642, 149048, 104337,
        252214, 282603, 298767, 265774, 266871, 168770, 263290]),
 array([0.6124176 , 0.61245417, 0.61245819, 0.61246286, 0.61244946,
        0.61247507, 0.61246538, 0.61247158, 0.61245924, 0.61245064,
        0.61245664, 0.6124409 , 0.612444  , 0.61245995, 0.61246149]))

In [278]:
#np.sum(1-features_bool[:,5])
np.sum(features[:,0]*features_bool[:,0])/np.sum(features_bool[:,0])
np.sum(1-features_bool[:,0])
#T_v[151800,0]

233870

In [239]:
features[2,0], 1

(1.4541874748240025e-05, 1)

In [210]:
#T_v[150000:150010,:3]

In [211]:
#cum_l_v[9999:10001,:5]

In [212]:
#l_curr_v[:20,:5]
whole_loss_v[-20:,:5]

array([[0.61248448, 0.61248416, 0.61248414, 0.61248151, 0.61248312],
       [0.61248445, 0.61248349, 0.61248445, 0.61248195, 0.6124826 ],
       [0.61248434, 0.61248377, 0.61248434, 0.61248237, 0.61248195],
       [0.61248415, 0.61248402, 0.61248415, 0.61248277, 0.61248115],
       [0.61248446, 0.61248422, 0.61248387, 0.61248315, 0.61248244],
       [0.61248436, 0.61248437, 0.61248346, 0.6124835 , 0.61248346],
       [0.61248446, 0.61248446, 0.61248292, 0.61248381, 0.61248292],
       [0.61248447, 0.61248447, 0.61248385, 0.61248407, 0.6124822 ],
       [0.61248439, 0.61248429, 0.61248338, 0.61248429, 0.61248125],
       [0.61248418, 0.61248443, 0.61248271, 0.61248443, 0.61248271],
       [0.6124838 , 0.61248448, 0.61248179, 0.61248448, 0.61248179],
       [0.61248321, 0.61248413, 0.61248321, 0.61248413, 0.61248321],
       [0.61248417, 0.61248435, 0.61248417, 0.61248435, 0.61248233],
       [0.61248447, 0.61248334, 0.61248447, 0.61248334, 0.61248102],
       [0.61248443, 0.61248378, 0.

In [10]:
y_v[:10, :10]

array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 1., 1.],
       [0., 0., 0., 0., 1., 1., 0., 1., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 1., 0., 0., 1.],
       [0., 0., 1., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

In [ ]:
for p in range(10000):
    for q in range(10):
        if label[axv[p, q]] != y_sub_v[p, q]:
            print('fail!!', p, q)
            break

In [ ]:
axv.shape

In [ ]:
axv[:3, :3]

In [ ]:
label[139807]

In [ ]:
features[:5, :5]